In [20]:
import pandas as pd
import numpy as np
from datetime import datetime
import sys
import os
import matplotlib.pyplot as plt
import json
import logging
from MSSQLDB_Datamanager import DataManager

## logger 設定
now = datetime.now()
log_filename = 'DBA_finproject_{}.log'.format(now.strftime('%Y-%m-%d'))
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
log_dir = '.\\var\\log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
file_handler = logging.FileHandler('./var/log/' + log_filename)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

## config 讀取
cfg_path = r".\config.json"
with open(cfg_path, 'r', encoding='utf-8-sig') as f:
    cfg = json.load(f)
# databse connection information
db_cfg = cfg['db_connect']
db_cfg['creator'] = __import__(db_cfg['creator'])

In [21]:
## 資料區間設定
time_start = '2023-01-01'
time_end = '2023-04-01'
## query 資料
DM = DataManager(logger, db_cfg ,time_start, time_end)
# 撈所有表
# Behavior,MemberData,OrderData,OrderSlave,SalePageData,SegmentData = DM.read_ALL_data_from_db()
# 只撈其中一張
OrderData = DM.read_data_from_db(TableName= 'OrderData')

{message : return db query result}


In [24]:
# 挑選要的欄位
OrderData = OrderData[['ShopMemberId', 'TradesGroupCode', 'OrderDateTime', 'StatusDef']].sort_values('OrderDateTime')
OrderData = OrderData[OrderData['StatusDef'] == 'Finish']
OrderData.reset_index(drop = True , inplace= True)
# 時間轉換
OrderData.OrderDateTime = OrderData.OrderDateTime.apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d %H:%M:%S'))
OrderData.OrderDateTime = pd.to_datetime(OrderData.OrderDateTime)
# 計算 Frequency
F = OrderData.groupby('ShopMemberId')['TradesGroupCode'].count().reset_index()
F.rename(columns= {'TradesGroupCode': 'Frequency'} , inplace=True)
# 找出不重複的id
member_FM = OrderData.copy()
member_FM.drop_duplicates('ShopMemberId', inplace= True)
member_FM = member_FM[['ShopMemberId']]
member_FM = member_FM.merge(F, on = 'ShopMemberId')
# 計算每個人的區間
interval = OrderData.copy()
# 計算每個人的購物間隔
interval['interval'] = interval.groupby('ShopMemberId', as_index = True).OrderDateTime.diff()
interval['interval'] = interval['interval'].dt.total_seconds() 
# 把只有一筆購物紀錄與第一筆購物紀錄刪掉
interval.dropna(inplace=True)
interval = interval.sort_values(['ShopMemberId', 'OrderDateTime']).groupby('ShopMemberId').interval.median().reset_index()
# 計算所有人購物間隔的中位數
overall_interval_median = interval.interval.median()
# 依會員mapping間隔中位數
member_FM = member_FM.merge(interval, on = 'ShopMemberId', how = 'left')
# 給予全部會員各自的間隔中位數與全體間隔中位數權重
member_FM['interval'] = member_FM.interval.apply(lambda x: abs(x + (overall_interval_median - x) * 0.5) if x is not None else x)
# 只購買一次的用全體中位數替代
member_FM.interval.fillna(overall_interval_median, inplace = True)
max_time = max(OrderData.OrderDateTime)
# 計算三倍購物周期的時間
member_FM['3PeriodTime'] = member_FM.interval.apply(lambda x: max_time - pd.Timedelta(seconds= 3 * x))
# 計算三倍購物週期期間的購物次數
times_in_three_period = pd.merge(member_FM, OrderData, how= 'left', on = 'ShopMemberId')
times_in_three_period['larger'] = times_in_three_period['OrderDateTime'] > times_in_three_period['3PeriodTime']
times_in_three_period = times_in_three_period.groupby('ShopMemberId')['larger'].agg(sum)
times_in_three_period = times_in_three_period.reset_index()
times_in_three_period.rename(columns = {'larger':'Recency'}, inplace=True)
member_FM = pd.merge(member_FM, times_in_three_period, on = 'ShopMemberId')
# NAPL 會員人數
N = member_FM[(member_FM.Frequency == 1)&(member_FM.Recency == 1)].ShopMemberId.tolist()
L = member_FM[(member_FM.Frequency == 1)&(member_FM.Recency == 0)].ShopMemberId.tolist()
P = member_FM[(member_FM.Frequency > 1)&(member_FM.Recency == 0)].ShopMemberId.tolist()
A =member_FM[(member_FM.Frequency > 1)&(member_FM.Recency >= 1)].ShopMemberId.tolist()
# Labeling
member_FM['Label'] = member_FM.ShopMemberId.apply(lambda x: 'N' if x in N else ('L' if x in L else ('P' if x in P else 'A')))

In [25]:
member_FM

,ShopMemberId,Frequency,interval,3PeriodTime,Recency,Label
0,w6obtv2EMtjB5NrvJh1J/iZklS3EU1Eaqlnf3YP57oE=,2,561118.0,2023-03-12 10:54:01.000,0,P
1,bydGLNp4kd03HMKSSOdYWKY1hXp0twBEJyhbxJbuD3s=,1,780373.0,2023-03-04 20:11:16.000,0,L
2,Sq+5qjoCnBeDMDrDJ4/fgYGwYJOB4n9NNUqjLoida8U=,1,780373.0,2023-03-04 20:11:16.000,0,L
3,k+UfauHVg1YqwquqkUMXhElH908TCiLJkB1YdVapJhU=,1,780373.0,2023-03-04 20:11:16.000,0,L
4,sqsqURBJCHF6M/WGsUGGy54VsBLikqrxYe9+YPnbzlY=,8,778655.5,2023-03-04 21:37:08.500,2,A
...,...,...,...,...,...,...
51669,tgfpQLs4gUNUTKTlwGsvnf+PgAQ8O04xMnxdrVYfcPc=,1,780373.0,2023-03-04 20:11:16.000,1,N
51670,xwRBkjj8Bf+lZXa22R+EfHgJUBOMatlbnIK3fBKHIg8=,1,780373.0,2023-03-04 20:11:16.000,1,N
51671,POWH/OqoIgMYknhtJ81k24+m84xtHR3D+CVW5z9RcPE=,1,780373.0,2023-03-04 20:11:16.000,1,N
51672,8ZPBd5wM6JceqVdt8tQvjTiXNm3w1FhH/3U+JfsF/Mo=,1,780373.0,2023-03-04 20:11:16.000,1,N
